In [14]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 2

In [ ]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
# gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
# beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0.1])  # shape (10,)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,2*min_degree,-2,2,0.1,-2,0.1,2,0.1,0]) # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 2


100%|██████████| 320/320 [00:01<00:00, 304.46it/s]


(0.6036183286432554,
 0.4164124999999999,
 0.42075,
 0.3439820610459721,
 0.36994524104748805)

In [16]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 200/200 [00:01<00:00, 103.54it/s]


{'average': 0.41311249999999994,
 'direct_effect': 0.27885000000000015,
 'spillover_effect': -0.1828625,
 'psi_1_gamma': 0.4988250000000001,
 'psi_0_gamma': 0.21997499999999995,
 'psi_zero': 0.40283749999999996}

In [17]:
from utils import run_pll
from run_pll import run_autognet

np.random.seed(0)
L_chain_noise = np.random.binomial(n=1, p=0.2, size=L_chain.shape)
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain_noise,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x12eef4e50> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 148.26it/s]

100%|██████████| 60/60 [00:00<00:00, 173.84it/s]


Multiprocessing finished.


In [18]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.52232 ± 0.02418
direct_effect: 0.57649 ± 0.04310
spillover_effect: -0.14115 ± 0.03816
psi_1_gamma: 0.69569 ± 0.03252
psi_0_gamma: 0.11920 ± 0.02686
psi_zero: 0.26035 ± 0.02676


{'average': 0.41311249999999994,
 'direct_effect': 0.27885000000000015,
 'spillover_effect': -0.1828625,
 'psi_1_gamma': 0.4988250000000001,
 'psi_0_gamma': 0.21997499999999995,
 'psi_zero': 0.40283749999999996}

In [19]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x12eef4e50> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 299.50it/s]


Multiprocessing finished.


In [20]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.41974 ± 0.03190
direct_effect: 0.28355 ± 0.04505
spillover_effect: -0.18712 ± 0.04567
psi_1_gamma: 0.50582 ± 0.03777
psi_0_gamma: 0.22227 ± 0.03752
psi_zero: 0.40939 ± 0.03809


In [21]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i,
#          'mispec': 'outcome'} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [22]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i,
#          'mispec': 'treatment'} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")